In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

%matplotlib inline
pd.options.display.float_format = '{:,.4f}'.format
plt.style.use('seaborn-white')

In [4]:
data = pd.read_csv('../data/prices_CBSA_vaca_construction_pop.csv')

/usr/local/anaconda3/envs/realty-market-app/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
data.head()

,CBSA,NAME,LSAD,POPESTIMATE2019,ZIP,RegionName,State,City,Metro,CountyName,...,ErrorMargin4,cbsa_code,CSA,Name,Total,unit_1,2 Units,unit_34,unit_5,unit_for_5orMore
0,10420,"Akron, OH",Metropolitan Statistical Area,703479,44256,44256,OH,Medina,Cleveland-Elyria,Medina County,...,10.1000,"10,420.0000",184.0000,"Akron, OH ...",945.0000,856.0000,2.0000,32.0000,55.0000,1.0000
1,10420,"Akron, OH",Metropolitan Statistical Area,703479,44256,44256,OH,Medina,Cleveland-Elyria,Medina County,...,10.1000,"10,420.0000",184.0000,"Akron, OH ...",945.0000,856.0000,2.0000,32.0000,55.0000,1.0000
2,10420,"Akron, OH",Metropolitan Statistical Area,703479,44256,44256,OH,Medina,Cleveland-Elyria,Medina County,...,10.1000,"10,420.0000",184.0000,"Akron, OH ...",945.0000,856.0000,2.0000,32.0000,55.0000,1.0000
3,10420,"Akron, OH",Metropolitan Statistical Area,703479,44256,44256,OH,Medina,Cleveland-Elyria,Medina County,...,10.1000,"10,420.0000",184.0000,"Akron, OH ...",945.0000,856.0000,2.0000,32.0000,55.0000,1.0000
4,10420,"Akron, OH",Metropolitan Statistical Area,703479,44256,44256,OH,Medina,Cleveland-Elyria,Medina County,...,10.1000,"10,420.0000",184.0000,"Akron, OH ...",945.0000,856.0000,2.0000,32.0000,55.0000,1.0000


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39756 entries, 0 to 39755
Data columns (total 31 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CBSA              39756 non-null  int64  
 1   NAME              39756 non-null  object 
 2   LSAD              39756 non-null  object 
 3   POPESTIMATE2019   39756 non-null  int64  
 4   ZIP               39756 non-null  int64  
 5   RegionName        39756 non-null  int64  
 6   State             39756 non-null  object 
 7   City              39756 non-null  object 
 8   Metro             39756 non-null  object 
 9   CountyName        39756 non-null  object 
 10  month_date        39756 non-null  object 
 11  house_price       39756 non-null  float64
 12  rent_price        38901 non-null  float64
 13  cbsa_title        36672 non-null  object 
 14  Q1                36672 non-null  float64
 15  ErrorMargin1      36672 non-null  float64
 16  Q2                36672 non-null  float6

In [12]:
# keep cols we want
data_to_agg = data.iloc[:, [0,2,1,4,6,10,11,12,3,16,25]].copy()

In [13]:
data_to_agg.head()

,CBSA,LSAD,NAME,ZIP,State,month_date,house_price,rent_price,POPESTIMATE2019,Q2,Total
0,10420,Metropolitan Statistical Area,"Akron, OH",44256,OH,2019-01,"241,373.0000","1,210.0000",703479,3.6000,945.0000
1,10420,Metropolitan Statistical Area,"Akron, OH",44256,OH,2019-02,"242,013.0000","1,234.0000",703479,3.6000,945.0000
2,10420,Metropolitan Statistical Area,"Akron, OH",44256,OH,2019-03,"242,620.0000","1,241.0000",703479,3.6000,945.0000
3,10420,Metropolitan Statistical Area,"Akron, OH",44256,OH,2019-04,"243,485.0000","1,270.0000",703479,3.6000,945.0000
4,10420,Metropolitan Statistical Area,"Akron, OH",44256,OH,2019-05,"244,326.0000","1,260.0000",703479,3.6000,945.0000


In [14]:
data_to_agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39756 entries, 0 to 39755
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CBSA             39756 non-null  int64  
 1   LSAD             39756 non-null  object 
 2   NAME             39756 non-null  object 
 3   ZIP              39756 non-null  int64  
 4   State            39756 non-null  object 
 5   month_date       39756 non-null  object 
 6   house_price      39756 non-null  float64
 7   rent_price       38901 non-null  float64
 8   POPESTIMATE2019  39756 non-null  int64  
 9   Q2               36672 non-null  float64
 10  Total            36672 non-null  float64
dtypes: float64(4), int64(3), object(4)
memory usage: 3.3+ MB


In [16]:
# do we have any nulls?
data_to_agg[data_to_agg.isna().any(axis=1)].count()
# thats ok, once we get means we should have rent data (some months there isn't any)

CBSA               3866
LSAD               3866
NAME               3866
ZIP                3866
State              3866
month_date         3866
house_price        3866
rent_price         3011
POPESTIMATE2019    3866
Q2                  782
Total               782
dtype: int64

In [21]:
data_to_agg = data_to_agg.rename(columns={'LSAD': 'CBSA_type',
                    'NAME': 'CBSA_name',
                    'ZIP' : 'zip_code',
                    'State' : 'state',
                    'POPESTIMATE2019': 'pop_2019_est',
                    'Q2' : 'vacancy_pct',
                    'Total' : 'construction_19_Q2'})

In [22]:
data_to_agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39756 entries, 0 to 39755
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CBSA                39756 non-null  int64  
 1   CBSA_type           39756 non-null  object 
 2   CBSA_name           39756 non-null  object 
 3   zip_code            39756 non-null  int64  
 4   state               39756 non-null  object 
 5   month_date          39756 non-null  object 
 6   house_price         39756 non-null  float64
 7   rent_price          38901 non-null  float64
 8   pop_2019_est        39756 non-null  int64  
 9   vacancy_pct         36672 non-null  float64
 10  construction_19_Q2  36672 non-null  float64
dtypes: float64(4), int64(3), object(4)
memory usage: 3.3+ MB


In [ ]:
# first aggregate by zip code for 2019 (so 12 months of data)
# then aggregate by CBSA, and track how many zips are included and what the SD is

agg_zip = data_to_agg.groupby('zip_code')